In [1]:
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pathlib
import joblib

import numpy as np
import matplotlib.pyplot as plt

import cv2
import PIL

import tensorflow as tf
import tensorflow_hub as hub

In [3]:
train_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Corn/train")
test_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Corn/test")
val_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Corn/val")

In [4]:
class_images = {
    "cercospora": list(train_dir.glob("cercospora/*")), 
    "healthy": list(train_dir.glob("healthy/*")),
    "northern_leaf_blight": list(train_dir.glob("northern_leaf_blight/*")),
    "common_rust": list(train_dir.glob("common_rust/*")),
}

class_labels = {
    "cercospora": 1,
    "healthy": 0,
    "northern_leaf_blight": 2,
    "common_rust": 3,
}


X, y = [], []

for image_name, images in class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(class_labels[image_name])
        
X = np.array(X)
y = np.array(y)

In [5]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [6]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(4)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(X, y, epochs=10)

Epoch 1/10
96/96 [==============================] - 13s 39ms/step - loss: 0.5370 - accuracy: 0.7741
Epoch 2/10
96/96 [==============================] - 4s 37ms/step - loss: 0.3547 - accuracy: 0.8529
Epoch 3/10
96/96 [==============================] - 4s 37ms/step - loss: 0.3170 - accuracy: 0.8651
Epoch 4/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2929 - accuracy: 0.8789
Epoch 5/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2739 - accuracy: 0.8828
Epoch 6/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2649 - accuracy: 0.8887
Epoch 7/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2582 - accuracy: 0.8923
Epoch 8/10
96/96 [==============================] - 4s 37ms/step - loss: 0.2450 - accuracy: 0.9015
Epoch 9/10
96/96 [==============================] - 4s 36ms/step - loss: 0.2339 - accuracy: 0.9054
Epoch 10/10
96/96 [==============================] - 3s 36ms/step - loss: 0.2258 - accuracy: 0.9110


# Evaluation

In [10]:
# on test

test_class_images = {
    "cercospora": list(test_dir.glob("cercospora/*")), 
    "healthy": list(test_dir.glob("healthy/*")),
    "northern_leaf_blight": list(test_dir.glob("northern_leaf_blight/*")),
    "common_rust": list(test_dir.glob("common_rust/*")),
}

test_class_labels = {
    "cercospora": 1,
    "healthy": 0,
    "northern_leaf_blight": 2,
    "common_rust": 3,
}


X_test, y_test = [], []

for image_name, images in test_class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X_test.append(resized_img)
        y_test.append(test_class_labels[image_name])
        
X_test = np.array(X_test)
y_test = np.array(y_test)

model.evaluate(X_test, y_test)

21/21 [==============================] - 2s 51ms/step - loss: 0.2671 - accuracy: 0.8702


[0.2671075165271759, 0.8702290058135986]

In [12]:
# on val

val_class_images = {
    "cercospora": list(val_dir.glob("cercospora/*")), 
    "healthy": list(val_dir.glob("healthy/*")),
    "northern_leaf_blight": list(val_dir.glob("northern_leaf_blight/*")),
    "common_rust": list(val_dir.glob("common_rust/*")),
}

val_class_labels = {
    "cercospora": 1,
    "healthy": 0,
    "northern_leaf_blight": 2,
    "common_rust": 3,
}


X_val, y_val = [], []

for image_name, images in val_class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X_val.append(resized_img)
        y_val.append(val_class_labels[image_name])
        
X_val = np.array(X_val)
y_val = np.array(y_val)

model.evaluate(X_val, y_val)

21/21 [==============================] - 1s 53ms/step - loss: 0.3106 - accuracy: 0.8721


[0.31056690216064453, 0.8721109628677368]

In [13]:
model.save("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/corn_model.h5", model)

In [14]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/corn_model.h5", custom_objects={'KerasLayer': hub.KerasLayer})

In [16]:
_dir = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/")

sample_images_list = list(_dir.glob("test images/*"))

for count in range(len(sample_images_list)):
  image = PIL.Image.open(str(sample_images_list[count]))
  image_array = np.array(image.resize((224, 224)))
  print(f"image: {str(sample_images_list[count])}",
        f"\nprediction: {np.argmax(loaded_model.predict(np.array([image_array])))}")

1/1 [==============================] - 0s 20ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/test images/image (813).jpg 
prediction: 0
1/1 [==============================] - 0s 19ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/test images/image (1015).JPG 
prediction: 3
1/1 [==============================] - 0s 18ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/test images/image (701).JPG 
prediction: 1
1/1 [==============================] - 0s 28ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/test images/image (706).JPG 
prediction: 1
